## 6.4. Interactive Dashboard

#### Importing Libraries


In [ ]:
import pandas as pd
import plotly.express as px
from dash import Input, Output, html, dcc, Dash
from IPython.display import VimeoVideo
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

#### Importing the Data 

In [ ]:
def wrangle(filepath):
    """
    Read SCF data file into ``DataFrame``.

    Returns only credit fearful households whose net worth is less than $2 million.

    Parameters
    ----------
    filepath : str
        Location of CSV file.
    """
    df = pd.read
